# Volunteer model loss calculation

This notebook takes a `subject_id` and calculates the MSE of each volunteer's model to the galaxy data. These losses are then glued to the notebook for use elsewhere.

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import json
import numpy as np
import pandas as pd
from tqdm import tqdm
import matplotlib.pyplot as plt
import seaborn as sns
from gzbuilder_analysis import parsing, rendering, fitting
import lib.galaxy_utilities as gu
import scrapbook as sb

/opt/apps/pkgs/anaconda3/2019.03/intel64/lib/python3.7/site-packages/dask/config.py:168: YAMLLoadWarning: calling yaml.load() without Loader=... is deprecated, as the default Loader is unsafe. Please read https://msg.pyyaml.org/load for full details.
  data = yaml.load(f.read()) or {}


In [3]:
subject_id = 20902040

In [4]:
# Parameters
subject_id = 21096846


In [5]:
diff_data_df = pd.read_pickle('lib/fitting_metadata.pkl')

In [6]:
def get_losses_by_user(subject_id):
  diff_data = diff_data_df.loc[subject_id]
  psf = diff_data['psf']
  pixel_mask = np.array(diff_data['pixel_mask'])[::-1]
  galaxy_data = np.array(diff_data['galaxy_data'])[::-1]
  image_size = galaxy_data.shape[0]
  size_diff = diff_data['size_diff']
  
  cls = gu.classifications.query('subject_ids == {}'.format(subject_id))
  names = cls.user_name 
  annotations = cls['annotations'].apply(json.loads)

  models = annotations.apply(
    parsing.parse_annotation,
    size_diff=size_diff
  )

  tqdm.pandas(
    desc='Rendering models'.format(subject_id),
    leave=False
  )
  rendered = models.progress_apply(
    rendering.calculate_model,
    image_size=image_size,
    psf=psf
  ).rename('rendered')

  tqdm.pandas(
    desc='Calculating differences'.format(subject_id),
    leave=False
  )
  losses = rendered.apply(
    fitting.loss,
    args=(galaxy_data, pixel_mask)
  ).rename('loss')
  all_losses = pd.Series(
    losses.values,
    index=names.values
  ).rename(subject_id)
  best_losses = pd.concat((
    all_losses.where(
      ~all_losses.duplicated(keep=False)
    ).dropna(), # all non-duplicates
    all_losses.where(
      all_losses.duplicated(keep=False)
    ).dropna().groupby(level=0).min() # best value of duplicates
  ), axis=0)
  return best_losses

In [7]:
losses = get_losses_by_user(subject_id)
losses

Rendering models:   0%|          | 0/31 [00:00<?, ?it/s]

Rendering models:   6%|▋         | 2/31 [00:05<01:20,  2.79s/it]

Rendering models:  13%|█▎        | 4/31 [00:05<00:53,  1.99s/it]

Rendering models:  19%|█▉        | 6/31 [00:06<00:35,  1.44s/it]

Rendering models:  23%|██▎       | 7/31 [00:07<00:35,  1.50s/it]

...

Rendering models:  90%|█████████ | 28/31 [00:12<00:00,  4.66it/s]

Rendering models:  94%|█████████▎| 29/31 [00:12<00:00,  3.59it/s]

Rendering models:  97%|█████████▋| 30/31 [00:12<00:00,  3.70it/s]

Rendering models: 100%|██████████| 31/31 [00:13<00:00,  4.07it/s]

not-logged-in-bcce6b83948dce684fbd    0.000232
not-logged-in-3b0b929f4475fd276561    0.006492
not-logged-in-3b0b929f4475fd276561    0.001866
framosuw                              0.006081
Idax98                                0.000122
HM01                                  0.000424
jnarayanbvg                           0.001143
Jonas_Cross                           0.000321
not-logged-in-c6917d9b19f41e6c2995    0.150206
vildkatt                              0.028967
lsautter                              0.000289
earthling001                          0.000262
not-logged-in-e931dfb2080048a803b3    0.000583
JulieHedglin                          0.000363
Planetme                              0.000755
Saharisunshine                        0.000152
acapirala                             0.002561
dinosora                              0.000214
not-logged-in-04ed9ae880f18762cd9e    0.074760
Lavadude                              0.003528
not-logged-in-0c9c201853b1aad7e203    0.026403
Anfrily      

In [8]:
sb.glue('subject_id', subject_id)
sb.glue('losses', losses.to_dict())